# [CMI-SleepState-Detection](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview)
## Child Mind Institute - Detect Sleep States
### Detect sleep onset and wake from wrist-worn accelerometer data
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- License: Private, Unlicensed, All the files in this repository under any branch are Prohibited to be used commercially or for personally, communally or privately unless permitted by author in writing.
- Copyrights 2023-2024 (c) are reserved only by the author: Najeeb Haider Zaidi
________________________________________________________________________
# Attributions:
The Dataset has been provided by Child Mind Institute. in [Kaggle Competition](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) which the author is participating in and authorized to use the dataset solely for the competition purposes.
________________________________________________________________________

In [2]:
!pip install pandarallel


In [1]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
#from pandarallel import pandarallel
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime as dts
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from tqdm import tqdm
tqdm.pandas()
#pandarallel.initialize(progress_bar=True)

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


In [ ]:
events=pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')

In [ ]:
df_full=pd.read_pickle('/kaggle/input/later-data/full_clustered .pkl')

# Analyzing the relationshiip between sleep and cluster

In [4]:
def inactive_periods(df):
    print("shape before application: ",df.shape)
    df['diff_anglez']=df['anglez'].diff()
    df=df[(df['enmo']!=0.0) | (df['diff_anglez']!=0.0)]
    print("shape after application: ",df.shape)
    df.drop('diff_anglez', inplace=True, axis=1)
    print("shape after completion: ",df.shape)
    print("removed ")
    return df
def clustering(df):
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler    
    X=df[['anglez','enmo']]
#Scalling the data
    scaler=StandardScaler()
    X_scaled=scaler.fit_transform(X)
#perform clustering
    model=KMeans(n_clusters=4,algorithm="elkan" )
    model.fit(X_scaled)
    return model.labels_
def rollingstd(series_df):
# Creating columns with nans
    series_df['sd_enmo_1']=np.nan    # 1 min rolling std: enmo
    series_df['sd_anglez_1']=np.nan  # 1 min rolling std: anglez
    series_df['m_enmo_2']=np.nan     # 2 min rolling mean: enmo
    series_df['m_anglez_2']=np.nan   # 2 min rolling std: anglez 
    print('anglez rolling std 12')
    series_df['sd_anglez_1'] = (series_df.groupby('series_id')['anglez']
                      .rolling(12)
                      .std()
                      .reset_index(level=0, drop=True))
    print('anglez rolling std 2')
    series_df['sd_anglez_1'][series_df['sd_anglez_1'].isna()==True] = (series_df.groupby('series_id')['anglez']
                      .rolling(2)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling std 12')
    series_df['sd_enmo_1'] = (series_df.groupby('series_id')['enmo']
                      .rolling(12)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling mean 24')
    series_df['m_enmo_2'] = (series_df.groupby('series_id')['enmo']
                      .rolling(24)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('anglez rolling mean 24')
    series_df['m_anglez_2'] = (series_df.groupby('series_id')['anglez']
                      .rolling(24)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('enmo rolling std 2')
    print('Nans in sd_emno_1: ',series_df['sd_enmo_1'].isnull().sum())
    series_df['sd_enmo_1'][series_df['sd_enmo_1'].isna()==True] = (series_df.groupby('series_id')['enmo']
                      .rolling(2)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling mean 2')
    series_df['m_enmo_2'][series_df['m_enmo_2'].isna()==True] = (series_df.groupby('series_id')['enmo']
                      .rolling(2)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('anglez rolling mean 2')
    series_df['m_anglez_2'][series_df['m_anglez_2'].isna()==True] = (series_df.groupby('series_id')['anglez']
                      .rolling(2)
                      .mean()
                      .reset_index(level=0, drop=True))
#Series wise rolling std and mean
# filling rest of nans
    print('Nans in sd_emno_1: ',series_df['sd_enmo_1'].isnull().sum())
    series_df['sd_enmo_1'].fillna(0.0, inplace=True)
    series_df['sd_anglez_1'].fillna(0.0, inplace=True)
    series_df['m_enmo_2'].fillna(0.0, inplace=True)
    series_df['m_anglez_2'].fillna(0.0, inplace=True)
    print('Nans after removal: ',series_df['sd_enmo_1'].isnull().sum())

    return(series_df)
def scale(X):
    from sklearn import preprocessing
    scaler=preprocessing.StandardScaler().fit(X)
    return (scaler.transform(X))


In [5]:
# Importing the datasets
print('Importing Training Datasets')
df_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')
df_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
print('Dataset Imported...')
print('______________________________________')

# Merging the datasets
print('Merging the training datasets...')
events=df_events[['series_id', 'step','event']]
series_df=pd.merge(df_series,events,on=["step","series_id"],how='left')
df_series=[]
series_df['sleep']=np.nan
series_df.loc[series_df["event"]=="onset", "sleep"] = 1
series_df.loc[series_df["event"]=="wakeup", "sleep"] = 0
series_df['sleep'].fillna(method='ffill', inplace=True)
series_df['sleep'].fillna(value=0, inplace=True)
print('Datasets Merged...')
print('______________________________________')

# Removing the periods of inactivity
print('Removing the periods of Inactivity...')
series_df=inactive_periods(series_df)
print('______________________________________')

# Forming Windows
win_size=720  #600mins
print('Creating Windows each size: ',win_size)
#series_df=window(series_df,win_size)
print('Windows formed...')
print('______________________________________')

# Adding the columns of Standard Deviation (1 min)
print('Adding columns to account for deviation in enmo and anglez 1 min rolling...')
series_df=rollingstd(series_df)
series_df['sd_anglez_1']=pd.to_numeric(series_df['sd_anglez_1'])
series_df['sd_enmo_1']=pd.to_numeric(series_df['sd_enmo_1'])
series_df['m_anglez_2']=pd.to_numeric(series_df['m_anglez_2'])
series_df['m_enmo_2']=pd.to_numeric(series_df['m_enmo_2'])
print('Std columns added...')
print('______________________________________')

# Clustering the Data
print('Clustering the data based on enmo and anglez...')
series_df['cluster']=(clustering(series_df)+1)/4
print('Added clusters...')


Importing Training Datasets
Dataset Imported...
______________________________________
Merging the training datasets...
Datasets Merged...
______________________________________
Removing the periods of Inactivity...
shape before application:  (127946340, 7)
shape after application:  (111766109, 8)
shape after completion:  (111766109, 7)
removed 
______________________________________
Creating Windows each size:  720
Windows formed...
______________________________________
Adding columns to account for deviation in enmo and anglez 1 min rolling...
anglez rolling std 12
anglez rolling std 2
enmo rolling std 12
enmo rolling mean 24
anglez rolling mean 24
enmo rolling std 2
Nans in sd_emno_1:  3047
enmo rolling mean 2
anglez rolling mean 2
Nans in sd_emno_1:  277
Nans after removal:  0
Std columns added...
______________________________________
Clustering the data based on enmo and anglez...
Added clusters...


In [6]:
from sklearn.model_selection import train_test_split
X=series_df[['sd_anglez_1','sd_enmo_1','anglez','m_anglez_2','m_enmo_2','enmo','cluster']]
y=series_df[['sleep']]
series_df=[]
X_scaled=scale(X)
X_train, X_test, y_train, y_test =train_test_split(X_scaled,y,test_size=0.2, random_state=42)


In [7]:
def evaluate(y_test,ypred):
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    print("F1 Score: ",f1_score(y_test,y_pred)) #mean of recall and precision
    cm = confusion_matrix(y_test, y_pred)
    figure= px.imshow(cm,text_auto=True, width=1200, height=1200)
    figure.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=-1,verbose=1) 
print ('Training the model')
rf.fit(X_train,y_train)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'rf_model.joblib')
y_pred=rf.predict(X_test)
evaluate(y_test,y_pred)

Training the model


In [ ]:
df_sleep=df.loc[df['sleep']==1]
df_onset=df.loc[df['sleep']==0]

In [ ]:
df_sleep['cluster'].mode()

In [ ]:
df_onset['cluster'].mode()

In [ ]:
#df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

In [ ]:
df.info()

In [ ]:
df.to_pickel('cluster_timestamped.pkl')

In [ ]:
events

In [ ]:
events['step'].isnull().sum()

In [ ]:
event_df=events.dropna()

In [ ]:
event_df

In [ ]:
df['window']=np.nan

In [ ]:
ind=df.index[df['event'].isna()==False]
print(ind)

In [ ]:
# 30 min window on change event, with single timestep:5sec. 
c=0
for i in tqdm(ind):
    a=i-360
    b=i+360
    df['window'].loc[a:i]=int(c)
    c=c+1
    df['window'].loc[i:b]=int(c)
    c=c+1
print('Windows Created: ', c)    



In [ ]:
df=df[df['window'].isna()==False]

In [ ]:
df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

# Adding rolling Deviation

In [ ]:
def prepare(df):
    df['sd_anglez_1']=pd.to_numeric(df['anglez'].rolling(window=12).std())
    df['sd_enmo_1']=pd.to_numeric(df['enmo'].rolling(window=12).std())
    df['sd_anglez_1'].fillna('0.0',inplace=True)
    df['sd_enmo_1'].fillna('0.0',inplace=True)
    df['sleep'] = df['sleep'].replace({0:False, 0.2:True})
    return(df)

## Analyzing the windows

In [ ]:
def visualization(df_n):
    for i in range(10):
        title="Sleep: "+ str(df_n['sleep'][df_n['window']==i].mean())
        fig=px.line(y=df_n['sleep'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i])
        fig.add_scatter(y=df_n['cluster'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='cluster')
        fig.add_scatter(y=df_n['enmo'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='sleep')
        fig.update_layout(title=title)
        fig.show()
visualization(df)

# Evaluate Prediction

In [ ]:
def evaluate(y_test,ypred):
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    print("F1 Score: ",f1_score(y_test,y_pred)) #mean of recall and precision
    cm=confusion_matrix(y_test, y_pred)
    figure= px.imshow(cm,text_auto=True, width=1200, height=1200)
    figure.show()

In [ ]:
df.groupby(df['sleep']).describe().transpose()

In [ ]:
#df.to_pickle('ts_win_60_diff.pkl')
path="/kaggle/input/60min-window-before-and-after-the-event/ts_win_60_diff-30.pkl"
df=prepare(pd.read_pickle(path))


In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X=df[['sd_anglez_1','sd_enmo_1','anglez','enmo','cluster']]
y=df[['sleep']]
#scaler=preprocessing.StandardScaler().fit(X)
#X_scaled = scaler.transform(X)
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
X_scaled

## Import Model

In [ ]:
f
y_pred=model.predict(X_test)
evaluate(y_test,y_pred)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=-1,verbose=1) 
print ('Training the model')
rf.fit(X_train,y_train)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'rf_model.joblib')
y_pred=rf.predict(X_test)
evaluate(y_test,y_pred)


# SVM Classifier

In [ ]:
from sklearn.svm import SVC
svm = SVC(verbose=1) # n_neighbors is the number of neighbors to consider when predicting a class label for
svm.fit(X_test,y_test)
y_pred=svm.predict(X_test)
evaluate(y_test,y_pred)

# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB() 
nb.fit(X_train,y_train)
#dump(nb, 'nb_model.joblib')
y_pred=nb.predict(X_test)
evaluate(y_test,y_pred)

# KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10,n_jobs=-1,verbose=1) # n_neighbors is the number of neighbors to consider when predicting a class label for
knn.fit(X_train,y_train)
#dump(knn, 'knn_model.joblib')
y_pred=knn.predict(X_test)
evaluate(y_test,y_pred)

In [ ]:
pred['predicted']=y_pred